In [1]:
from env import get_db_url
import pandas as pd
import acquire
import prepare

In [2]:
df = acquire.acquire_data()

In [3]:
df = prepare.prepare_superstore(df)

In [4]:
df.columns

Index(['region_id', 'product_id', 'customer_id', 'category_id', 'order_id',
       'order_date', 'ship_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'sales', 'quantity', 'discount', 'profit',
       'category', 'sub-category', 'customer_name', 'product_name',
       'region_name'],
      dtype='object')

In [13]:
df.city.value_counts()

New York City    192
Los Angeles      124
Philadelphia     123
San Francisco     96
Seattle           62
                ... 
Carlsbad           1
Redlands           1
Whittier           1
Santa Clara        1
Brownsville        1
Name: city, Length: 247, dtype: int64

In [11]:
df.order_date.sort_values()

977    2014-01-04
978    2014-01-04
979    2014-01-04
942    2014-01-09
941    2014-01-09
          ...    
952    2017-12-30
1453   2017-12-30
1452   2017-12-30
1451   2017-12-30
637    2017-12-30
Name: order_date, Length: 1734, dtype: datetime64[ns]

In [12]:
df.sales

0        261.9600
1        731.9400
2        957.5775
3         22.3680
4         15.5520
          ...    
1729      47.7440
1730     434.3520
1731       3.5520
1732      88.8320
1733    1048.3500
Name: sales, Length: 1734, dtype: float64